This code is experimental and provided as is. Comments can be emailed to tony.bruguier@gmail.com

You need to download your PG&E data. This is the current instructions: https://www.pge.com/pge_global/common/pdfs/save-energy-money/analyze-your-usage/energy-data-hub/Download-My-Data-User-Guide.pdf

Be sure to use the option "Export usage for a range of days" so that you have hour-by-hour usage. There is a one-year limit so, if you want a longer period, you will have to repeat the process and concatenate the files. Otherwise, be sure to have an overlap with the irradiation data. I have my own usage data here.

In [7]:
import csv
from datetime import (date, datetime, time, timedelta)
from IPython.core.display import HTML

In [2]:
usage_filename = 'pge_usage.csv'  # Point to your file that you downloaded.

# The PG&E data takes into account daylight saving time, but the solar
# data (below) does not. So we convert everything to winter time. You might
# have to change the initial value of 'in_winter_time' because I didn't have
# the heart to handle timezone.
in_winter_time = False

prev_dt = None
usage_data = {}
with open(usage_filename, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        if len(row) >= 7 and row[0] == 'Electric usage':
            d = date.fromisoformat(row[1])
            t = time.fromisoformat(row[2])
                
            if prev_dt:
                time_delta_hours = (datetime.combine(d, t) - prev_dt).seconds / 3600
                
                if time_delta_hours == 0:
                    if in_winter_time:
                        raise ValueError('Initial value of in_winter_time should have been False')
                    in_winter_time = True
                elif time_delta_hours == 2:
                    if not in_winter_time:
                        raise ValueError('Initial value of in_winter_time should have been True')
                    in_winter_time = False
                else:
                    assert time_delta_hours == 1
            
            u = float(row[4])
            
            dt = datetime.combine(d, t) 
            dt_corrected = dt if in_winter_time else dt - timedelta(seconds=3600)
            
            assert dt_corrected not in usage_data, '%s already inserted' % (dt)
            usage_data[dt_corrected] = (u, d.weekday(), t.hour)
            
            prev_dt = dt

Now, you can download the solar irradiation data. It is available there: https://nsrdb.nrel.gov/data-sets/download-instructions.html

The code currently uses the approximation of using the GHI because I suspect it's more conservative.

There appears to be more fancy measures:
"Photovoltaic system derived data for determining the solar resource and
for modeling the performance of other photovoltaic systems" by Bill Marion and Benjamin Smith.
https://www.osti.gov/pages/servlets/purl/1349802

In [3]:
solar_filename = 'solar_data.csv'  # Point to your file that you downloaded.

ghi_index = -1
solar_data = {}
with open(solar_filename, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        if ghi_index == -1:
            ghi_index = row.index('GHI')
        else:
            d = date(int(row[0]), int(row[1]), int(row[2]))
            # We ignore the minutes, and just add every value for a given hour.
            t = time(int(row[3]), 0)
            dt = datetime.combine(d, t)
            
            s = float(row[ghi_index])
            
            solar_data[dt] = solar_data.get(dt, 0.0) + s

# Technically, we should take into account leap years, but the government somehow forgot that February 29 exists.
num_years_solar_data = len(solar_data) / (365.0 * 24.0)

I make another approximation. Vendors typically give us an amount of power that will be generated during a year. So what I do is compute a linearity coefficient between the energy (kW) that the vendor will give us and the DHI (kWh / m^2). This allows me to estimate the amount of power generated for any day and any time.

In [4]:
power_advertised = 5221  # [kWh / year]
print('Average power advertised: %.0f W' % (power_advertised / (365.0 * 24.0) * 1000.0))

# I assume that the vendor is lying about the energy produced, so if you have 0.80 below, it means that the
# system will only deliver 80% of what was advertised.
vendor_lying_factor = 0.80 # []

yearly_solar_irradiation = sum(solar_data.values()) / num_years_solar_data  # [kWh / m^2 / year]

irradiation_to_power = power_advertised * vendor_lying_factor / yearly_solar_irradiation  # [m^2]

Average power advertised: 596 W


I now compute the amount of power we expect to buy and sell on weekdays and weekend days, for each hour. Note that for a given slot, we could both be buying and selling. For example, on a Sunday at 1pm we might be selling power (not home), but the next Sunday at 1pm we might be buying (it's raining and we stayed home). It doesn't really matter, as long as electricity is bought and sold at the same price on Sundays at 1pm.

In [18]:
start_analysis_date = date.fromisoformat('2019-01-01')

generated_weekdays = [0.0] * 24
generated_weekends = [0.0] * 24
consumed_weekdays = [0.0] * 24
consumed_weekends = [0.0] * 24

date = start_analysis_date
while date < start_analysis_date + timedelta(days=365):
    if date.month == 2 and date.day == 29:
        # We ignore leap years, since we don't have data anyway.
        continue
        
    for hour in range(24):
        dt = datetime.combine(date, time(hour, 0))
        
        usage, day_of_week, civil_hour = usage_data[dt]
        generated = solar_data[dt] * irradiation_to_power
        
        if day_of_week in [0, 1, 2, 3, 4]:
            if usage > generated:
                consumed_weekdays[civil_hour] += usage - generated
            else:
                generated_weekdays[civil_hour] += generated - usage
        else:
            if usage > generated:
                consumed_weekends[civil_hour] += usage - generated
            else:
                generated_weekends[civil_hour] += generated - usage
        
    date += timedelta(days=1)

html = '<table style="width:100%"><tr><th>Hour</th><th>Weekday consumed</th><th>Weekday generated</th><th>Weekend consumed</th><th>Weekend generated</th></tr>'
for hour in range(24):
    html += '<tr><td>%d</td><td>%.0f kWh</td><td>%.0f kWh</td><td>%.0f kWh</td><td>%.0f kWh</td></tr>' % (hour, consumed_weekdays[hour], generated_weekdays[hour], consumed_weekends[hour], generated_weekends[hour])
html += '</table>'
    
HTML(html)



Hour,Weekday consumed,Weekday generated,Weekend consumed,Weekend generated
0,56 kWh,0 kWh,24 kWh,0 kWh
1,58 kWh,0 kWh,27 kWh,0 kWh
2,59 kWh,0 kWh,24 kWh,0 kWh
3,57 kWh,0 kWh,24 kWh,0 kWh
4,60 kWh,0 kWh,25 kWh,0 kWh
5,67 kWh,0 kWh,27 kWh,0 kWh
6,89 kWh,0 kWh,27 kWh,0 kWh
7,56 kWh,9 kWh,24 kWh,3 kWh
8,7 kWh,75 kWh,17 kWh,13 kWh
9,5 kWh,166 kWh,21 kWh,43 kWh


I discount the future cash flow by using a risk-free interest. This could come either from:
* buying EE Bonds (guaranteed to double in 20 years, https://www.treasurydirect.gov/indiv/products/prod_eebonds_glance.htm)
* buying 20 year Treasurys (current rates: https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield)
* pre-paying the mortgage